In [2]:
import os
from groq import Groq
import pandas as pd
import json

client = Groq(api_key="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")

def gen_content(prompt):
    completion = client.chat.completions.create(model="mixtral-8x7b-32768", temperature=1.0, max_tokens=32768, top_p=0.5, stream=False, stop=None,
                                                messages=[
                                                            {
                                                                "role": "system",
                                                                "content": "You are a helpful assistant, which generates invalid questions from paragraphs."
                                                            },
                                                            {
                                                                "role": "user",
                                                                "content": prompt,
                                                            }
                                                         ])

    return completion.choices[0].message.content

def format_questions(text):
    # Split the text into individual questions based on the numbering
    questions = text.split('? ')

    # Remove the numbering from each question and combine them with commas
    formatted_questions = ', '.join([q.split('. ')[1] for q in questions if q.strip()])

    return formatted_questions

df = pd.read_csv('new_data.csv')
paragraphs = df['Paragraphs'].unique().tolist()

In [3]:
import re
from tqdm import tqdm

invs = []
for item in tqdm(paragraphs, total=len(paragraphs)):
    prompt = f"Generate 10 invalid questions based on Bloom's Taxonomy Remembering category from this given paragraph - {item}. These questions should not having any valid answer available in this paragraph."
    out = gen_content(prompt)
    data = out.split('\n')
    for item in data:
        text = '\n'.join([re.sub(r'^\d+\.\s+', '', line) for line in item.split('\n')])
        invs.append(text)

df['invalids'] = invs
df.to_csv('df.csv')

  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [01:16<00:00,  7.70s/it]


In [11]:
###    Checking for the validity of questions by answering ..
def gen_content(prompt):
    completion = client.chat.completions.create(model="mixtral-8x7b-32768", temperature=1.0, max_tokens=32768, top_p=0.5, stream=False, stop=None,
                                                messages=[
                                                            {
                                                                "role": "system",
                                                                "content": "You are a helpful assistant, which checks that a question should be answered from a paragraph or not.."
                                                            },
                                                            {
                                                                "role": "user",
                                                                "content": prompt,
                                                            }
                                                         ])

    return completion.choices[0].message.content

responses = []
messages = []

data = pd.read_csv('df.csv')
for item in tqdm(data.iterrows(), total=df.shape[0]):
    question = item[1]['question']
    context = item[1]['Paragraphs']
    prompt = f"Read this paragraph {context} and the given question {question}, and check whether this question can be answered from this paragraph or not and answer in 'yes' or 'no'"
    out = gen_content(prompt)
    response = out.split(', ')[0]
    message = out.split(', ')[1:]
    responses.append(response)
    messages.append(message)



100%|██████████| 100/100 [08:00<00:00,  4.81s/it]


In [12]:
df['responses'] = responses
df['messages'] = messages
df

,Unnamed: 0,question,category,Paragraphs,invalids,responses,messages
0,0,What was the name of the town where Mr. Aberna...,Remembering,"In the small town of Meadowgrove, a mysterious...",What is Mr. Abernathy's first name?,Yes,[the question can be answered from the paragra...
1,1,Who were the key figures in the town's decisio...,Remembering,"In the small town of Meadowgrove, a mysterious...",Where did Mr. Abernathy work before moving to ...,No,[this question cannot be answered from the par...
2,2,How did the townsfolk initially react to the s...,Understanding,"In the small town of Meadowgrove, a mysterious...",How many weeks did it take for the town's chil...,Yes,[the question can be answered from the paragra...
3,3,What were the town's elders initially suspicio...,Understanding,"In the small town of Meadowgrove, a mysterious...",What is the exact address of the mansion where...,Yes,[the question can be answered from the paragra...
4,4,Why did Mr. Abernathy choose to reside in the ...,Understanding,"In the small town of Meadowgrove, a mysterious...","What type of warlock is Mr. Abernathy, and wha...",Yes,[the question can be answered from the paragra...
...,...,...,...,...,...,...,...
95,95,How did Emily's adventures change her perspect...,Remembering,Emily's adventures inspired a new generation o...,What is the breed of the dog that Emily had as...,No,[the question cannot be answered from the para...
96,96,What lessons did Emily learn from her adventures?,Remembering,Emily's adventures inspired a new generation o...,What is the favorite food of the archaeologist...,No,[this question cannot be answered from the par...
97,97,What kind of adventures did Emily believe were...,Remembering,Emily's adventures inspired a new generation o...,What is the middle name of Emily?,No,[this question cannot be answered from the par...
98,98,What drove Emily to continue exploring and dis...,Remembering,Emily's adventures inspired a new generation o...,What is the eye color of the new generation of...,No,[this question cannot be answered directly fro...


In [24]:
df = pd.read_csv("df2.csv")
df

,Unnamed: 0.1,Unnamed: 0,question,category,Paragraphs,invalids,responses,messages
0,0,0,What was the name of the town where Mr. Aberna...,Remembering,"In the small town of Meadowgrove, a mysterious...",What is Mr. Abernathy's first name?,Yes,['the question can be answered from the paragr...
1,1,1,Who were the key figures in the town's decisio...,Remembering,"In the small town of Meadowgrove, a mysterious...",Where did Mr. Abernathy work before moving to ...,No,"[""this question cannot be answered from the pa..."
2,2,2,How did the townsfolk initially react to the s...,Understanding,"In the small town of Meadowgrove, a mysterious...",How many weeks did it take for the town's chil...,Yes,['the question can be answered from the paragr...
3,3,3,What were the town's elders initially suspicio...,Understanding,"In the small town of Meadowgrove, a mysterious...",What is the exact address of the mansion where...,Yes,['the question can be answered from the paragr...
4,4,4,Why did Mr. Abernathy choose to reside in the ...,Understanding,"In the small town of Meadowgrove, a mysterious...","What type of warlock is Mr. Abernathy, and wha...",Yes,['the question can be answered from the paragr...
...,...,...,...,...,...,...,...,...
95,95,95,How did Emily's adventures change her perspect...,Remembering,Emily's adventures inspired a new generation o...,What is the breed of the dog that Emily had as...,No,"[""the question cannot be answered from the par..."
96,96,96,What lessons did Emily learn from her adventures?,Remembering,Emily's adventures inspired a new generation o...,What is the favorite food of the archaeologist...,No,['this question cannot be answered from the pa...
97,97,97,What kind of adventures did Emily believe were...,Remembering,Emily's adventures inspired a new generation o...,What is the middle name of Emily?,No,['this question cannot be answered from the pa...
98,98,98,What drove Emily to continue exploring and dis...,Remembering,Emily's adventures inspired a new generation o...,What is the eye color of the new generation of...,No,"[""this question cannot be answered directly fr..."


In [25]:
df = df[df['responses'] == 'No']
df.to_csv('final1.csv')